In [55]:
import pandas as pd
import os
import ast
from openai import OpenAI
from langchain.prompts import PromptTemplate

In [41]:
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

In [42]:
prompt_template = PromptTemplate.from_template(
    '''
    Your task is to extract keywords from {sentence}.
    You are an assistant for labeling negative annotaded keywords in user reviews.
    Return the words that are assumed to be negative words in form of a list.
    Note: If now negative words detected reutrn empty list.
    Note: Words have to stand alone as negative to be included. No fill words.
    Note: Only words that are explicit in the sentence should be included.
    Provide output without further text information. Use the following schema ['keyword 1', 'keyword 2', ...]
    '''
)


In [43]:
client = OpenAI()

In [44]:
def extract_keywords(sentence: str) -> str:
    try:
        # Format the prompt dynamically with the input sentence
        prompt = prompt_template.format(sentence=sentence)
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

In [45]:
df = pd.read_csv('../data/raw/reviews_en_goerlitzerpark.csv')

In [46]:
snippet = df.copy()
snippet = snippet[0:10]

In [47]:
############# maybe kill stop words in before

In [48]:
snippet['keywords'] = snippet['review_text_english'].apply(lambda x: extract_keywords(x))

In [53]:
snippet.keywords[0]

"['tipsy', 'drugs', 'homeless', 'rubbish']"

In [56]:
ast.literal_eval(snippet.keywords[0])

['tipsy', 'drugs', 'homeless', 'rubbish']